## Laboratorio SQL y Python

In [182]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors



#### Tabla Foursquare

In [102]:
df_fs = pd.read_csv("datos/api_foursquare.csv", index_col=0)
df_fs.head()


,municipio,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude
0,ajalvir,Park,4d3c6c5b84d46ea87dddfd5c,LikelyOpen,563.0,/v3/places/4d3c6c5b84d46ea87dddfd5c,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341
1,ajalvir,Park,4f54f7bfe4b036244d02685b,VeryLikelyOpen,1724.0,/v3/places/4f54f7bfe4b036244d02685b,Parque Infantil,NaN,40.543867,-3.464773
2,ajalvir,Park,7d4ef99ca2dd4789f032a90c,LikelyOpen,333.0,/v3/places/7d4ef99ca2dd4789f032a90c,Chiquitin,Ajalvir Madrid,40.531662,-3.482835
3,alamo-el,Park,53de1144498e8fd8d8cb3537,LikelyOpen,118.0,/v3/places/53de1144498e8fd8d8cb3537,Isla Perejil,28607 El Alamo Comunidad de Madrid,40.230993,-3.991942
4,alamo-el,Park,51a8f451498ecb8334de37df,LikelyOpen,582.0,/v3/places/51a8f451498ecb8334de37df,Parque de Madera,NaN,40.234636,-3.995166


#### Tabla AEMET

In [103]:
df_aemet = pd.read_csv("datos/df_aemet_final.csv")
df_aux = df_aemet["localizacion_id"].str.split("-id", expand=True)
df_aemet = pd.concat([df_aemet, df_aux], axis=1)
df_aemet.drop(columns=["localizacion_id"], inplace=True)
df_aemet.rename(columns={0: "municipio", 1: "id"}, inplace=True)

ultima_columna = df_aemet.columns[-1]
penultima_columna = df_aemet.columns[-2]
df_aemet = df_aemet[[ultima_columna, penultima_columna] + list(df_aemet.columns[:-2])]

df_aemet

,id,municipio,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento
0,28001,acebeda-la,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7
1,28001,acebeda-la,2024-08-29 13:00:00,Nubes altas,24,24,29.0,0.0,0,57,Riesgo,0,75,Riesgo,SE,12
2,28001,acebeda-la,2024-08-29 14:00:00,Muy nuboso,23,23,33.0,0.0,0,59,100,0,80,Riesgo,S,14
3,28001,acebeda-la,2024-08-29 15:00:00,Nubes altas,23,23,35.0,0.0,0,55,Riesgo,0,80,Riesgo,S,17
4,28001,acebeda-la,2024-08-29 16:00:00,Nubes altas,21,21,34.0,0.0,0,65,Riesgo,0,80,Riesgo,S,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7880,28183,zarzalejo,2024-08-31 03:00:00,Poco nuboso,17,17,13.0,0.0,0,90,5,0,5,Sin riesgo,N,9
7881,28183,zarzalejo,2024-08-31 04:00:00,Cielo despejado,17,17,14.0,0.0,0,91,5,0,5,Sin riesgo,NE,8
7882,28183,zarzalejo,2024-08-31 05:00:00,Cielo despejado,17,17,13.0,0.0,0,88,5,0,5,Sin riesgo,N,9
7883,28183,zarzalejo,2024-08-31 06:00:00,Poco nuboso,16,16,10.0,0.0,0,89,5,0,5,Sin riesgo,N,8


#### Tabla municipios

In [104]:
df_municipios = pd.read_csv("datos/municipios.csv", index_col=0).T
df_municipios.rename(columns={0: "latitude", 1: "longitude", 2:"municipio"}, inplace=True)
df_municipios = df_municipios.reset_index().drop(columns=["municipio"]).rename(columns={"index":"municipio"})
df_municipios.head()

,municipio,latitude,longitude
0,acebeda-la,41.0869583,-3.624399
1,ajalvir,40.5342302,-3.4807818
2,alameda-del-valle,40.9177178,-3.8438216
3,alamo-el,40.2307296,-3.9905893
4,alcala-de-henares,40.4818396,-3.3644973


Queremos añadir a la tabla municipios el id de cada municipio que lo sacamos de la tabla de la aemet:

In [105]:
len(df_municipios["municipio"].unique())

176

In [106]:
df = df_aemet[["id", "municipio"]]
df_aux = df.drop_duplicates()

l1=len(df_municipios["municipio"].unique())
l2 = len(df_aux["municipio"].unique())

print(f"En df_municipios tenemos {l1} municipios únicos")
print(f"En df_aemet tenemos {l2} municipios únicos")

En df_municipios tenemos 176 municipios únicos
En df_aemet tenemos 179 municipios únicos


Los municipios de los cuales no tenemos la latitud y longitud porque no estan en la tabla municipios los pasamos por geopy y los añadimos al df municipios. Luego hacemos un merge con la subtabla de la aemet y asi obtenemos los indices.

In [107]:
l_muni = list(set(df_aux["municipio"].unique()) - set(df_municipios["municipio"].unique()))

geolocator = Nominatim(user_agent="SetMagic")
lista_dics = []

for lugar in l_muni:
    location = geolocator.geocode(lugar)
    dic_aux={
        "municipio":lugar,
        "latitude":location.latitude,
        "longitude":location.longitude
    }
    lista_dics.append(dic_aux)

df_muni_aux=pd.DataFrame(lista_dics)
df_municipios= pd.concat([df_municipios, df_muni_aux], ignore_index=True)
df_municipios.tail()

,municipio,latitude,longitude
174,villavieja-del-lozoya,41.0063622,-3.6716291
175,zarzalejo,40.5483016,-4.1815899
176,molinos-los,6.483333,-72.65
177,leganes,10.785528,122.588814
178,cabrera-la,13.347245,144.758651


In [108]:
df_municipios = pd.merge(df_aux, df_municipios)
print(df_municipios.shape)
df_municipios.head()

(179, 4)


,id,municipio,latitude,longitude
0,28001,acebeda-la,41.0869583,-3.624399
1,28002,ajalvir,40.5342302,-3.4807818
2,28003,alameda-del-valle,40.9177178,-3.8438216
3,28004,alamo-el,40.2307296,-3.9905893
4,28005,alcala-de-henares,40.4818396,-3.3644973


En la df de foursquare tendremos que añadir el id por municipio:

In [121]:
df_muni_id = df_municipios[["id", "municipio"]]

In [124]:
df_fs = df_fs.merge(right=df_muni_id, how='left', on ='municipio')
df_fs.head()

,municipio,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude,id
0,ajalvir,Park,4d3c6c5b84d46ea87dddfd5c,LikelyOpen,563.0,/v3/places/4d3c6c5b84d46ea87dddfd5c,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341,28002
1,ajalvir,Park,4f54f7bfe4b036244d02685b,VeryLikelyOpen,1724.0,/v3/places/4f54f7bfe4b036244d02685b,Parque Infantil,NaN,40.543867,-3.464773,28002
2,ajalvir,Park,7d4ef99ca2dd4789f032a90c,LikelyOpen,333.0,/v3/places/7d4ef99ca2dd4789f032a90c,Chiquitin,Ajalvir Madrid,40.531662,-3.482835,28002
3,alamo-el,Park,53de1144498e8fd8d8cb3537,LikelyOpen,118.0,/v3/places/53de1144498e8fd8d8cb3537,Isla Perejil,28607 El Alamo Comunidad de Madrid,40.230993,-3.991942,28004
4,alamo-el,Park,51a8f451498ecb8334de37df,LikelyOpen,582.0,/v3/places/51a8f451498ecb8334de37df,Parque de Madera,NaN,40.234636,-3.995166,28004


_____

Comprobamos que nulos hay en las tablas y vemos que solo hay nulos en la de df_fs en el campo de direcciones, rellenamos las direcciones faltante:

In [141]:
df_fs["address"].isnull().sum()

np.int64(129)

In [152]:
df_fs.head()

,municipio,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude,id
0,ajalvir,Park,4d3c6c5b84d46ea87dddfd5c,LikelyOpen,563.0,/v3/places/4d3c6c5b84d46ea87dddfd5c,diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341,28002
1,ajalvir,Park,4f54f7bfe4b036244d02685b,VeryLikelyOpen,1724.0,/v3/places/4f54f7bfe4b036244d02685b,parque infantil,NaN,40.543867,-3.464773,28002
2,ajalvir,Park,7d4ef99ca2dd4789f032a90c,LikelyOpen,333.0,/v3/places/7d4ef99ca2dd4789f032a90c,chiquitin,Ajalvir Madrid,40.531662,-3.482835,28002
3,alamo-el,Park,53de1144498e8fd8d8cb3537,LikelyOpen,118.0,/v3/places/53de1144498e8fd8d8cb3537,isla perejil,28607 El Alamo Comunidad de Madrid,40.230993,-3.991942,28004
4,alamo-el,Park,51a8f451498ecb8334de37df,LikelyOpen,582.0,/v3/places/51a8f451498ecb8334de37df,parque de madera,NaN,40.234636,-3.995166,28004


In [176]:
def get_direccion(dir, lat, long):
    geolocator = Nominatim(user_agent="lab")
    if type(dir)==float:
        location = geolocator.reverse(""+str(lat)+", "+str(long))
        return location.address
    else:
        return dir

In [177]:
df_fs["address"] = df_fs.apply(lambda row: get_direccion(row["address"], row["latitude"], row["longitude"]), axis=1)

In [180]:
df_fs["address"].isnull().sum()

np.int64(0)

____

### Preparamos las tablas que vamos a insertar en la base de datos:

Df municipio:

In [204]:
df_municipios.rename(columns={"id":"id_municipio", "latitude":"latitud", "longitude":"longitud"}, inplace=True)
df_municipios.head()

,id_municipio,municipio,latitud,longitud
0,28001,acebeda-la,41.0869583,-3.624399
1,28002,ajalvir,40.5342302,-3.4807818
2,28003,alameda-del-valle,40.9177178,-3.8438216
3,28004,alamo-el,40.2307296,-3.9905893
4,28005,alcala-de-henares,40.4818396,-3.3644973


Df del cielo:

In [201]:
df_cielo = pd.DataFrame(df_aemet["cielo"].unique())
df_cielo = df_cielo.reset_index()
df_cielo.rename(columns={"index":"id_cielo", 0:"nombre"}, inplace=True)
df_cielo["id_cielo"] = df_cielo["id_cielo"].apply(lambda x: x+1)
df_cielo.head()

,id_cielo,nombre
0,1,Nubes altas
1,2,Muy nuboso
2,3,Cubierto con tormenta y lluvia escasa
3,4,Nuboso
4,5,Cielo despejado


Df tiempo:

In [210]:
df_tiempo = df_aemet.drop(columns=["municipio", "prob._precip.__(%)", "avisos"])
print(df_tiempo.shape)
df_tiempo.head(1)

(7885, 13)


,id,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._de_nieve_(%),prob._de_tormenta_(%),dirección_viento,velocidad_del_viento
0,28001,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,0,75,S,7


In [311]:
df_merged_tiempo = df_tiempo.merge(right=df_cielo, how='left', left_on ='cielo', right_on="nombre")
df_merged_tiempo.drop(columns=["cielo", "nombre"], inplace=True)
df_merged_tiempo.columns
df_tiempo2 = df_merged_tiempo.reindex(columns=['id', 'fecha', 'id_cielo', 'temp._(°c)', 'sen._térmica_(°c)', 'racha_máx._(km/h)',
       'precipitación_(mm)', 'nieve_(mm)', 'humedad_relativa_(%)',
       'prob._de_nieve_(%)', 'prob._de_tormenta_(%)', 'dirección_viento',
       'velocidad_del_viento'])

In [312]:
df_fechas = df_tiempo2["fecha"].str.split(expand=True).rename(columns={0:"fecha", 1:"hora"})
df_tiempo2.drop(columns=["fecha"], inplace=True)
df_tiempo2 = pd.concat([df_tiempo2, df_fechas], axis=1)
df_tiempo2=df_tiempo2.reindex(columns=['id', 'fecha', 'hora', 'id_cielo', 'temp._(°c)', 'sen._térmica_(°c)',
       'racha_máx._(km/h)', 'precipitación_(mm)', 'nieve_(mm)',
       'humedad_relativa_(%)', 'prob._de_nieve_(%)', 'prob._de_tormenta_(%)',
       'dirección_viento', 'velocidad_del_viento'])
df_tiempo2.head()

,id,fecha,hora,id_cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._de_nieve_(%),prob._de_tormenta_(%),dirección_viento,velocidad_del_viento
0,28001,2024-08-29,12:00:00,1,21,21,25.0,0.0,0,69,0,75,S,7
1,28001,2024-08-29,13:00:00,1,24,24,29.0,0.0,0,57,0,75,SE,12
2,28001,2024-08-29,14:00:00,2,23,23,33.0,0.0,0,59,0,80,S,14
3,28001,2024-08-29,15:00:00,1,23,23,35.0,0.0,0,55,0,80,S,17
4,28001,2024-08-29,16:00:00,1,21,21,34.0,0.0,0,65,0,80,S,20


Df de los lugares de foursquare:

In [ ]:
df_fs.drop(columns=["municipio", "fsq_id"], inplace=True)

In [253]:
df_fs.shape

(907, 9)

Creamos el df de apertura:

In [242]:
df_apertura = pd.DataFrame(df_fs["closed_bucket"].unique(), columns=["nombre"])
df_apertura = df_apertura.reset_index().rename(columns={"index":"id_apertura"})
df_apertura["id_apertura"] = df_apertura["id_apertura"].apply(lambda x: x+1)
df_apertura

,id_apertura,nombre
0,1,LikelyOpen
1,2,VeryLikelyOpen
2,3,Unsure


Creamos el df de categoria:

In [245]:
df_categoria = pd.DataFrame(df_fs["category"].unique(), columns=["nombre"])
df_categoria = df_categoria.reset_index().rename(columns={"index":"id_categoria"})
df_categoria["id_categoria"] = df_categoria["id_categoria"].apply(lambda x: x+1)
df_categoria.head(1)

,id_categoria,nombre
0,1,Park


Ahora en la tabla principal sustituimos los campos de categoria y apertura por sus índices correspondientes:

In [256]:
df_merged_fs = df_fs.merge(right=df_apertura, how='left', left_on ='closed_bucket', right_on="nombre")
df_merged_fs_2 = df_merged_fs.merge(right=df_categoria, how='left', left_on ='category', right_on="nombre")
df_merged_fs_2=df_merged_fs_2.drop(columns=["category", "closed_bucket", "nombre_x", "nombre_y"])
df_merged_fs_2.columns

Index(['distance', 'link', 'name', 'address', 'latitude', 'longitude', 'id',
       'id_apertura', 'id_categoria'],
      dtype='object')

In [261]:
df_lugares = df_merged_fs_2.reindex(columns=['id', 'id_categoria', 'id_apertura', 'distance', 'name', 'address', 'latitude', 'longitude', 'link'])
df_lugares.head(1)

,id,id_categoria,id_apertura,distance,name,address,latitude,longitude,link
0,28002,1,1,563.0,diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341,/v3/places/4d3c6c5b84d46ea87dddfd5c


Nuestros 6 dataframes listos para insertaren la base de datos son:

In [ ]:
# df_municipios

# df_tiempo2
# df_cielo

# df_lugares
# df_apertura
# df_categoria

________________

### Nos conectamos a la bbdd y creamos las tablas:

In [328]:
try:  
    conexion = psycopg2.connect(
        database = "municipios",
        user = "postgres",
        password = "admin",
        host = "localhost",
        port = "5432")
    
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("La contraseña es erronea")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexion")
    else:
        print(f"Ocurrió el error {e}")

cursor = conexion.cursor()

In [317]:
# Creación tabla municipios
query_creacion_municipio = """create table if not exists municipios (
                        id_municipio int primary key,
                        nombre varchar(100) not null,
                        latitud numeric(12,8) not null,
                        longitud numeric(12,8) not null
                        );"""

cursor.execute(query_creacion_municipio)
conexion.commit()


# Creación tabla estado_cielo
query_creacion_cielo = """create table if not exists estado_cielo (
                        id_cielo int primary key,
                        nombre varchar(100) not null
                        );"""

cursor.execute(query_creacion_cielo)
conexion.commit()


# Creación tabla tiempo
query_creacion_tiempo = """create table if not exists tiempo (
                        id_tiempo serial primary key,
                        id_municipio int not null,
                        fecha date not null,
                        hora time not null,
                        id_cielo int not null,
                        temperatura_grados int not null,
                        sensacion_termica_grados int not null,
                        racha_maxima_km_h numeric(6,2) not null,
                        precipitacion_mm numeric(5,2) not null,
                        nieve_mm numeric(5,2) not null,
                        humedad_relativa_porcentaje int not null check(humedad_relativa_porcentaje < 101),
                        probabilidad_nieve_porcentaje int not null check(humedad_relativa_porcentaje < 101),
                        probabilidad_tormenta_porcentaje int not null check(humedad_relativa_porcentaje < 101),
                        direccion_viento varchar(2) not null,
                        velocidad_viento_km_h int not null,
                        foreign key (id_municipio) references municipios(id_municipio) on delete restrict on update cascade,
	                    foreign key (id_cielo) references estado_cielo(id_cielo) on delete restrict on update cascade
                        );"""

cursor.execute(query_creacion_tiempo)
conexion.commit()

# Creación tabla categoria
query_creacion_categoria = """create table if not exists categoria (
                        id_categoria int primary key,
                        nombre varchar(100) not null
                        );"""

cursor.execute(query_creacion_categoria)
conexion.commit()


# Creación tabla horario_apertura
query_creacion_horario_apertura = """create table if not exists horario_apertura (
                        id_apertura int primary key,
                        nombre varchar(100) not null
                        );"""

cursor.execute(query_creacion_horario_apertura)
conexion.commit()


# Creación tabla lugares
query_creacion_tiempo = """create table if not exists lugares (
                        id_lugar serial primary key,
                        id_municipio int not null,
                        id_categoria int not null,
                        id_apertura int not null,
                        distancia numeric(7,2) not null,
                        nombre varchar(100) not null,
                        direccion varchar(300) not null,
                        latitud numeric(12,8) not null,
                        longitud numeric(12,8) not null,
                        link varchar(1000) not null,
                        foreign key (id_municipio) references municipios(id_municipio) on delete restrict on update cascade,
	                    foreign key (id_categoria) references categoria(id_categoria) on delete restrict on update cascade,
                        foreign key (id_apertura) references horario_apertura(id_apertura) on delete restrict on update cascade
                        );"""

cursor.execute(query_creacion_tiempo)
conexion.commit()


### Insertamos los datos

In [318]:
# Insertamos los datos en la tabla municipios
lista_tuplas_municipios = [tuple(fila) for fila in df_municipios.values]
query_insercion = "insert into municipios (id_municipio, nombre, latitud, longitud) values (%s, %s, %s, %s)"
cursor.executemany(query_insercion, lista_tuplas_municipios)
conexion.commit()

In [319]:
# Insertamos los datos en la tabla estado_cielo
lista_tuplas_cielo = [tuple(fila) for fila in df_cielo.values]
query_insercion = "insert into estado_cielo (id_cielo, nombre) values (%s, %s)"
cursor.executemany(query_insercion, lista_tuplas_cielo)
conexion.commit()

In [323]:
# Insertamos los datos en la tabla tiempo
lista_tuplas_tiempo = [tuple(fila) for fila in df_tiempo2.values]
query_insercion = """insert into tiempo 
(id_municipio, fecha, hora, id_cielo, temperatura_grados, sensacion_termica_grados, racha_maxima_km_h, precipitacion_mm, nieve_mm, humedad_relativa_porcentaje, probabilidad_nieve_porcentaje, probabilidad_tormenta_porcentaje, direccion_viento, velocidad_viento_km_h) 
values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
cursor.executemany(query_insercion, lista_tuplas_tiempo)
conexion.commit()

In [324]:
# Insertamos los datos en la tabla categorias
lista_tuplas_categoria = [tuple(fila) for fila in df_categoria.values]
query_insercion = "insert into categoria (id_categoria, nombre) values (%s, %s)"
cursor.executemany(query_insercion, lista_tuplas_categoria)
conexion.commit()

In [325]:
# Insertamos los datos en la tabla horario_apertura
lista_tuplas_horario_apertura = [tuple(fila) for fila in df_apertura.values]
query_insercion = "insert into horario_apertura (id_apertura, nombre) values (%s, %s)"
cursor.executemany(query_insercion, lista_tuplas_horario_apertura)
conexion.commit()

In [329]:
# Insertamos los datos en la tabla lugares
lista_tuplas_lugares = [tuple(fila) for fila in df_lugares.values]
query_insercion = """insert into 
lugares (id_municipio, id_categoria, id_apertura, distancia, nombre, direccion, latitud, longitud, link) 
values (%s, %s, %s, %s, %s, %s, %s, %s, %s)"""
cursor.executemany(query_insercion, lista_tuplas_lugares)
conexion.commit()

In [280]:
cursor.close()
conexion.close()